In [78]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid, save_image

print('PyTorch version:', torch.__version__)
print('torchvision version:', torchvision.__version__)
use_gpu = torch.cuda.is_available()
print('Is GPU available:', use_gpu)

PyTorch version: 1.0.0
torchvision version: 0.2.1
Is GPU available: True


In [45]:
# general settings
device = torch.device('cuda' if use_gpu else 'cpu')

BATCHSIZE = 64

data_dir = '../../data/CelebA/celeba-64x64-images-npy/'

output_dir = '../../data/CelebA/celeba-64x64-outputs/'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
save_dir = '../../data/CelebA/celeba-64x64-save/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [46]:
files = os.listdir(data_dir)
for i, f in enumerate(files):
    if i == 0:
        train_images = np.load(data_dir + f)
    else:
        train_images = np.concatenate((train_images, np.load(data_dir + f)))

In [47]:
train_loader = DataLoader((torch.from_numpy(train_images).float() - 127.5) / 127.5, batch_size=BATCHSIZE, shuffle=True)

In [69]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [73]:
class Generator(nn.Module):
    def __init__(self, nz, nglf):
        super(Generator, self).__init__()
        self.ct1 = nn.ConvTranspose2d(nz, nglf*8, kernel_size=4, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(nglf*8)
        self.rl1 = nn.ReLU(inplace=True)
        
        self.ct2 = nn.ConvTranspose2d(nglf*8, nglf*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(nglf*4)
        self.rl2 = nn.ReLU(inplace=True)
        
        self.ct3 = nn.ConvTranspose2d(nglf*4, nglf*2, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(nglf*2)
        self.rl3 = nn.ReLU(inplace=True)
        
        self.ct4 = nn.ConvTranspose2d(nglf*2, nglf, kernel_size=4, stride=2, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(nglf)
        self.rl4 = nn.ReLU(inplace=True)
        
        self.ct5 = nn.ConvTranspose2d(nglf, 3, kernel_size=4, stride=2, padding=1, bias=False)
        self.th5 = nn.Tanh()
        
    def forward(self, z):
        out = self.rl1(self.bn1(self.ct1(z)))
        out = self.rl2(self.bn2(self.ct2(out)))
        out = self.rl3(self.bn3(self.ct3(out)))
        out = self.rl4(self.bn4(self.ct4(out)))
        out = self.th5(self.ct5(out))
        return out

In [75]:
class Discriminator(nn.Module):
    def __init__(self, ndff):
        super(Discriminator, self).__init__()
        self.cv1 = nn.Conv2d(3, ndff, 4, 2, 1, bias=False)
        self.lr1 = nn.LeakyReLU(0.2, inplace=True)
        
        self.cv2 = nn.Conv2d(ndff, ndff*2, 4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(ndff*2)
        self.lr2 = nn.LeakyReLU(0.2, inplace=True)
        
        self.cv3 = nn.Conv2d(ndff*2, ndff*4, 4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(ndff*4)
        self.lr3 = nn.LeakyReLU(0.2, inplace=True)
        
        self.cv4 = nn.Conv2d(ndff*4, ndff*8, 4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(ndff*8)
        self.lr4 = nn.LeakyReLU(0.2, inplace=True)
        
        self.cv5 = nn.Conv2d(ndff*8, 1, 4, 1, 0, bias=False)
        self.sg5 = nn.Sigmoid()
        
    def forward(self, x):
        out = self.lr1(self.cv1(x))
        out = self.lr2(self.bn2(self.cv2(out)))
        out = self.lr3(self.bn3(self.cv3(out)))
        out = self.lr4(self.bn4(self.cv4(out)))
        out = self.sg5(self.cv5(out))
        return out.view(-1, 1).squeeze(1)

In [81]:
netG = Generator(100, 64).to(device)
netD = Discriminator(64).to(device)

netG.apply(weight_init)
netD.apply(weight_init)

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable